In [123]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import csv
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
import re

In [124]:
sss=pd.read_csv('/Users/Ari/desktop/ironhack/-Proyec-1-Pandas-Shark-attacks-/attacks.csv', encoding_errors='replace')
s_original=sss.copy()


# 1. LECTURA E INFO DE LA TABLA.

Lo primero que realizo es una "investigación" rapida sobre la información que me proporciona la tabla para saber que pasos seguir a posteriori.


In [125]:
sss.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [126]:
sss.shape

(25723, 24)

In [127]:
sss.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [128]:
sss.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

# 2. Proceso de limpieza. Eliminacion de tablas y duplicados

-Lo primero que voy a hacer es cambiar un poco los nombres de las columnas para que sea mmás sencillo trabajar con ellas...luego cuando acabe las dejaré algo mejor.

-Como se puede observar en la info de arriba hay bastantes nulos en especial en las columnas unnamed...en las que tan solo hay un valor que no sea nulo. Esas columnas no me ofrecen ningun tipo de información por lo que voy a proceder a borrarlas.


In [129]:
#es cambiar el nombre de las coumnas para que me cojan todo
sss.columns=[c.lower().replace(' ', '_').replace(':','').replace('.','') for c in sss.columns]
sss.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex_', 'age', 'injury', 'fatal_(y/n)', 'time',
       'species_', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number1', 'case_number2', 'original_order', 'unnamed_22',
       'unnamed_23'],
      dtype='object')

In [130]:
##lo primero borro esos valores que tienen tan solo 1 o 2 no nulos...osea unnamed 22 y 23
#se crea una variable llamada nan_cols  que nos da la suma de los valores nulos de cada coluumna, despues vemos los valores que
#tienen valores nulos mayores de cero 

nan_cols=sss.isna().sum()
#nan_cols[nan_cols>0]

In [131]:
sss.unnamed_23.unique() ##veo el contenido para asegurarme, de 22 tb

array([nan, 'Teramo', 'change filename'], dtype=object)

In [132]:

nan_cols[nan_cols>2.3e4].index


Index(['unnamed_22', 'unnamed_23'], dtype='object')

In [133]:
sss.drop(columns=nan_cols[nan_cols>2.3e4].index, inplace=True)

In [134]:
sss.shape 

(25723, 22)

# 2.1 Eliminación de duplicados...

In [135]:
sss.drop_duplicates().shape==sss.shape#esto dice que si que hay muchos duplicados

False

In [136]:
sss=sss.drop_duplicates()# elimino los duplicados

In [137]:
dim_original=s_original.shape#se me habia olvidado hacer esto

In [138]:
sss.shape, dim_original#comparo, mucas filas menos

((6312, 22), (25723, 24))

In [139]:
sss2=sss.copy()#creo copia por si 



# 2.2 Eliminación de una gran cantidad de valore nulos 

-Me doy cuenta que las 10 ultimas filas de todas las columnas están llenas de valores nulos que no me valen tampoco para nada, por lo que las elimino

In [140]:
sss.tail(10)

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,pdf,href_formula,href,case_number1,case_number2,original_order
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6304.0
6303,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6305.0
6304,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6306.0
6305,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6307.0
6306,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6308.0
6307,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6309.0
6308,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6310.0
6309,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25722,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
sss.drop(sss.tail(10).index, inplace=True)
sss.head()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,pdf,href_formula,href,case_number1,case_number2,original_order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0


In [142]:
sss.shape, dim_original

((6302, 22), (25723, 24))

# 2.3 Tablas semejantes.

-Observo que hay cuatro tablas que se asemejan mucho  case_number 1 case_number2 case_number y Date, valoro los duplicados que hay en unas y otros y me dispongo a borrar las que no me aporten mucha más información. En este caso me quedaré solo con la columna case_number, que es la que más info aporta y la más sencilla de modificar

-Lo mismo ocurre con las tablas href y href formula, investigo cuales son las no duplicadas y por qué y decido borrar una de ellas ya que la única diferencia es que en los no duplicados de href fórmula está el link repetido

In [143]:
(sss.drop_duplicates().case_number1==sss.case_number2).value_counts()
##el valor de no duplicados es irrisorio, por lo que me puedo permitir el eliminar columnas

True     6282
False      20
dtype: int64

In [144]:
(sss.drop_duplicates().case_number1==sss.case_number).value_counts()
##con esto igual

True     6278
False      24
dtype: int64

In [145]:
sss3=sss.copy()

In [146]:
sss=sss.drop(columns=['case_number1', 'case_number2'])

In [147]:
sss.shape, dim_original##ya estan borradas

((6302, 20), (25723, 24))

In [148]:
(sss.drop_duplicates().href_formula==sss.href).head()


0    True
1    True
2    True
3    True
4    True
dtype: bool

In [149]:
sss.href_formula[141]


'http://sharkattackfile.net/spreadsheets/pdf_directory/2017.05.27-Selwood.pdf'

In [150]:
sss.href[141]

'http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2017.05.27-Selwood.pdf'

In [151]:
sss=sss.drop(columns=['href'])

In [152]:
sss.shape, dim_original 

((6302, 19), (25723, 24))

In [153]:
#sss.info(memory_usage='deep')

In [154]:
sss=sss.drop(columns=['date'])

In [155]:
sss.shape, dim_original 

((6302, 18), (25723, 24))

# 3.Proceso limpieza columna case_number:

-Estas son las columnas con las que me voy a quedar, aunque no me interesan todas para mi hipotesis no veo motivos para borrarlas por lo que empiezo  a trabajar con cada una de ellas.

-La columna case_number tiene valores en diferentes formatos, quiero equipararlos todos. Para ello primero la paso a string para poder aplicarle la funcion y luego como la función no ha hecho todos los cambios que yo quería, reviso y modifico uno a uno los valores que no me gustan.

In [156]:
for case_number in sss.select_dtypes('object').columns:
    
    sss[case_number]=sss[case_number].astype('category')
    
sss.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6302 entries, 0 to 6301
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   case_number             6301 non-null   category
 1   year                    6300 non-null   float64 
 2   type                    6298 non-null   category
 3   country                 6252 non-null   category
 4   area                    5847 non-null   category
 5   location                5762 non-null   category
 6   activity                5758 non-null   category
 7   name                    6092 non-null   category
 8   sex_                    5737 non-null   category
 9   age                     3471 non-null   category
 10  injury                  6274 non-null   category
 11  fatal_(y/n)             5763 non-null   category
 12  time                    2948 non-null   category
 13  species_                3464 non-null   category
 14  investigator_or_source  

In [157]:
#creo una funcion para limpiar los date
patron='\d+.\d+.\d+'
def timebad(x):
    cont=0
    num=re.findall(patron,x)
    num=''.join(num)
    if num=='':
        cont=0
        
    elif num in x:
        cont+=1
        
    if cont==0:
        return 'unknown'
    elif cont==1:
        return str(num)

In [158]:
sss.case_number=sss.case_number.apply(timebad)

In [159]:
sss.at[6174,'case_number']='unknown'

In [160]:
mal=sss.case_number[sss.case_number.isna()].index
mal

Int64Index([5488], dtype='int64')

In [161]:
sss.at[6175,'case_number']='unknown'
sss.at[6176,'case_number']='unknown'
sss.at[6177,'case_number']='unknown'
sss.at[6178,'case_number']='unknown'
sss.at[6179,'case_number']='unknown'
sss.at[6180,'case_number']='unknown'
sss.at[5488,'case_number']='unknown'

In [162]:
#reviso por ultima vez que la lista case_number esta bien
sss.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6302 entries, 0 to 6301
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   case_number             6302 non-null   object  
 1   year                    6300 non-null   float64 
 2   type                    6298 non-null   category
 3   country                 6252 non-null   category
 4   area                    5847 non-null   category
 5   location                5762 non-null   category
 6   activity                5758 non-null   category
 7   name                    6092 non-null   category
 8   sex_                    5737 non-null   category
 9   age                     3471 non-null   category
 10  injury                  6274 non-null   category
 11  fatal_(y/n)             5763 non-null   category
 12  time                    2948 non-null   category
 13  species_                3464 non-null   category
 14  investigator_or_source  

## 4. Proceso limpieza columna year:
-Tan solo existen dos valores nulos en esta columna...no me interesa ponerle unknown xq son solo dos por los que los relleno equiparandolos a los valores contiguos.

-No me gusta que algunos valores tengan cero, por lo que creo una función para cambiarlos por Unknown.

In [163]:
mal=sss.year[sss.year.isna()].index 
mal

Int64Index([187, 6079], dtype='int64')

In [164]:
sss['year']=sss['year'].bfill()

In [165]:
sss.year.head()

0    2018.0
1    2018.0
2    2018.0
3    2018.0
4    2018.0
Name: year, dtype: float64

In [166]:
#for year in sss.select_dtypes('float').columns:
    
    #sss[case_number]=sss[case_number].astype('int')
    
#sss.info(memory_usage='deep')

In [167]:
#nan_cols=sss.isna().sum()
#nan_cols[nan_cols>0]

In [168]:
sss['year']=sss['year'].astype('int') #pasado a int para poder modificarlo
    
sss.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6302 entries, 0 to 6301
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   case_number             6302 non-null   object  
 1   year                    6302 non-null   int32   
 2   type                    6298 non-null   category
 3   country                 6252 non-null   category
 4   area                    5847 non-null   category
 5   location                5762 non-null   category
 6   activity                5758 non-null   category
 7   name                    6092 non-null   category
 8   sex_                    5737 non-null   category
 9   age                     3471 non-null   category
 10  injury                  6274 non-null   category
 11  fatal_(y/n)             5763 non-null   category
 12  time                    2948 non-null   category
 13  species_                3464 non-null   category
 14  investigator_or_source  

In [169]:
#sss.year.value_counts().head()

In [170]:
def years(x):
        if x==0:
            return 'unkwnon'
        else:
            return x
        
        #funcion de transformacion
    

In [171]:
sss.year=sss.year.apply(years)

In [172]:
sss.year.head() #acabado

0    2018
1    2018
2    2018
3    2018
4    2018
Name: year, dtype: object

# 5. Proceso de limpieza columna type:

-La modificacion de categorias anterior me ha modificado todas las categorias de las columnas, asi que esto tengo que pasarlo a string pra que me deje modificarlo...por lo que los valores nulos pasan a convertirse en una string y los tengo que modificar como tales, además aprovecho y modifico un poco más los valores unicos ya que hay muchos que se repiten, los boat.

In [173]:
nan_cols=sss.isna().sum()
nan_cols[nan_cols>0]

type                         4
country                     50
area                       455
location                   540
activity                   544
name                       210
sex_                       565
age                       2831
injury                      28
fatal_(y/n)                539
time                      3354
species_                  2838
investigator_or_source      17
href_formula                 1
dtype: int64

In [174]:
#el type no me lo coge asi que cambio nombre
sss.rename(columns = {'type':'typess'}, inplace = True)

In [175]:
#sss.columns

In [176]:
sss['typess']=sss['typess'].astype('str') #pasado a str para poder modificarlo
    
sss.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6302 entries, 0 to 6301
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   case_number             6302 non-null   object  
 1   year                    6302 non-null   object  
 2   typess                  6302 non-null   object  
 3   country                 6252 non-null   category
 4   area                    5847 non-null   category
 5   location                5762 non-null   category
 6   activity                5758 non-null   category
 7   name                    6092 non-null   category
 8   sex_                    5737 non-null   category
 9   age                     3471 non-null   category
 10  injury                  6274 non-null   category
 11  fatal_(y/n)             5763 non-null   category
 12  time                    2948 non-null   category
 13  species_                3464 non-null   category
 14  investigator_or_source  

In [177]:
sss.typess.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', 'nan', 'Boat', 'Boatomg'], dtype=object)

In [178]:
sss.typess.replace(['nan', 'Boating','Boatomg'], ['Unknown', 'Boat','Boat'],inplace=True)

In [179]:
sss.typess.unique()

array(['Boat', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', 'Unknown'], dtype=object)

# 6. Proceso limpieza columna country:

-Modifico los valores nulos por UNKNOWN y hago una serie de modificaciones para que no tengan carateres raros y estén todos en mayuúsculas.

In [180]:

nan_cols=sss.isna().sum()
nan_cols[nan_cols>0]   

country                     50
area                       455
location                   540
activity                   544
name                       210
sex_                       565
age                       2831
injury                      28
fatal_(y/n)                539
time                      3354
species_                  2838
investigator_or_source      17
href_formula                 1
dtype: int64

In [181]:
sss4=sss.copy()

In [182]:
for country in sss.select_dtypes('category').columns:
    
    sss[country]=sss[country].astype('object')
    
sss.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6302 entries, 0 to 6301
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             6302 non-null   object 
 1   year                    6302 non-null   object 
 2   typess                  6302 non-null   object 
 3   country                 6252 non-null   object 
 4   area                    5847 non-null   object 
 5   location                5762 non-null   object 
 6   activity                5758 non-null   object 
 7   name                    6092 non-null   object 
 8   sex_                    5737 non-null   object 
 9   age                     3471 non-null   object 
 10  injury                  6274 non-null   object 
 11  fatal_(y/n)             5763 non-null   object 
 12  time                    2948 non-null   object 
 13  species_                3464 non-null   object 
 14  investigator_or_source  6285 non-null   

In [183]:
sss.country.fillna('UNKNOWN', inplace=True) #relleno los nulos con unknown

In [184]:
sss.country= sss.country.str.replace('?', '')#quito los caracteres que no me gustan....ose una interrogacion


In [185]:
sss['country']=sss['country'].str.upper()#trnasformo en mayusculas


# 7. Proceso limpieza columna area:

-Esta columna tiene 455 valores nulos que voy a sustituir por unknown unicamente.

In [186]:
nan_cols=sss.isna().sum()
nan_cols[nan_cols>0]

area                       455
location                   540
activity                   544
name                       210
sex_                       565
age                       2831
injury                      28
fatal_(y/n)                539
time                      3354
species_                  2838
investigator_or_source      17
href_formula                 1
dtype: int64

In [187]:
#area
sss.area.fillna('Unknown', inplace=True)

In [188]:
sss.area.head() 

0         California
1            Georgia
2             Hawaii
3    New South Wales
4             Colima
Name: area, dtype: object

# 8.Proceso limpieza columna Location:

-Solo modifico los valores nulos por unknown

In [189]:
#location, a estas no pienso hacer nada
sss.location.fillna('Unknown', inplace=True)

In [190]:
nan_cols=sss.isna().sum()
nan_cols[nan_cols>0]

activity                   544
name                       210
sex_                       565
age                       2831
injury                      28
fatal_(y/n)                539
time                      3354
species_                  2838
investigator_or_source      17
href_formula                 1
dtype: int64

# 9.Proceso de limpieza columna Activity:

-La columna activity tiene 544 nulos que voy a sustituir por unknown y muchísimos valores unicos que se pueden agrupar para poder reducirlos.


In [191]:

sss.activity.fillna('Unknown', inplace=True)

In [192]:
nan_cols=sss.isna().sum()
nan_cols[nan_cols>0]

name                       210
sex_                       565
age                       2831
injury                      28
fatal_(y/n)                539
time                      3354
species_                  2838
investigator_or_source      17
href_formula                 1
dtype: int64

In [193]:
len(sss["activity"].unique())

1532

In [194]:
def cleanact(x):
    dicc_actividades = {"Fishing":re.search(".*[Ff](ishing|ISHING).*",str(x)),
                    "Swimming":re.search(".*[Ss](wimming|WIMMing).*",str(x)),
                    "Kite":re.search(".*[Kk](ite|ITE).*",str(x)),
                    "Walking":re.search(".*[Ww](alking|ALKING).*",str(x)),
                    "Boogie Board":re.search(".*[Bb](oogie|OOGIE).*",str(x)),
                    "Body Boarding":re.search(".*[Bb](ody|ODY).*",str(x)),
                    "Wind Surfing":re.search(".*[wW](ind|IND).*",str(x)),
                    "Boat":re.search(".*[Bb](oat|OAT).*",str(x)),
                    "Interact with sharks":re.search(".*[Ss](hark|HARK).*",str(x)),
                    "Diving":re.search(".*[Dd](iving|IVING).*",str(x)),
                    "Standing in water":re.search(".*[Ss](tand|TAND).*",str(x)),
                    "Paddling":re.search(".*[Pp](addl|ADDL).*",str(x)),
                    "Bathing":re.search(".*[Bb](athing|ATHING).*",str(x)),
                    "OverBoard":re.search(".*[Oo](verb|VERB).*",str(x)),
                    "Bathing":re.search(".*[Bb](athing|ATHING).*",str(x)),
                    "Floating":re.search(".*[Ff](loat|LOAT).*",str(x)),
                    "Jumping":re.search(".*[Jj](ump|UMP).*",str(x))}

    
    for key,values in dicc_actividades.items():
        if values:
            return key
    return "other"

In [195]:
sss["activity"] = sss["activity"].apply(cleanact)

In [196]:
sss.activity.unique()

array(['Paddling', 'Standing in water', 'other', 'Diving', 'Kite',
       'Swimming', 'Fishing', 'Walking', 'Interact with sharks',
       'Boogie Board', 'Body Boarding', 'Wind Surfing', 'Boat', 'Bathing',
       'Floating', 'Jumping', 'OverBoard'], dtype=object)

# 10. Proceso limpieza columna name:

-En esta columna voy a sustituir los valores nulos por unknown.
-Mientras realizo este proceso me doy cuenta que hay muchos valores que pone female o male y los modifico por unknown. Más tarde me doy cuenta de que ademas de female o male hay muchos valores que ponen otras cosas...por lo que aplico otra funcion porteriormente(pese a que igual podría haberlo hecho en una) que me modifique todos los valores que no empiecen por mayúsculas por unknonw...

In [197]:
#name:
#name tiene muchoos nulos y muchos female y male
sss.name.fillna('Unknown', inplace=True)

In [198]:
nan_cols=sss.isna().sum()
nan_cols[nan_cols>0]

sex_                       565
age                       2831
injury                      28
fatal_(y/n)                539
time                      3354
species_                  2838
investigator_or_source      17
href_formula                 1
dtype: int64

In [199]:
def nombres(x):
    
        if x=='male':
            return 'Unkwnon'
        elif x=='female':
            return 'Unkwnon'   
        else:
            return x

In [200]:
sss["name"] = sss["name"].apply(nombres)

In [201]:
def nombres(name):
        if re.search("^[A-Z].+",str(name)):
            return name
        else:
            return 'unknown'

In [202]:
sss["name"] = sss["name"].apply(nombres)

In [203]:
sss.name.head()

0        Julie Wolfe
1    Adyson�McNeely 
2        John Denges
3            Unkwnon
4     Gustavo Ramos 
Name: name, dtype: object

# 11.Proceso limpieza columna sex:

-Cambio los valores nulos por unknown.

-Reemplazo los valores unicos para que solo me den F, M o Unknown.

In [204]:
sss.sex_.fillna('Unknown', inplace=True)

In [205]:
sss.sex_.unique()

array(['F', 'M', 'Unknown', 'M ', 'lli', 'N', '.'], dtype=object)

In [206]:
sss.sex_.replace(['F','M','Unknown','M ','lli','N','.'],
                        ['F','M','Unknown','M','Unknown','Unknown','Unknown'],
                        inplace=True)

In [207]:
sss.sex_.unique()

array(['F', 'M', 'Unknown'], dtype=object)

# 12.Proceso de limpieza age:

-Age tiene 2800 valores nulos que voy a sustituit por unknown.

-Después procedere a sustituir los valores que estan mal por unknown o por lo que cosidere...tendría que haberlo hecho con una funcion ya que resultaron ser bastantes más de los que creía.

In [208]:
sss.age.unique()

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', '� ', ' ', '30 or 36', '6�', '21 & ?', '75',
       '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28',
       '20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly',
       'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50',
       '18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 35',
     

In [209]:
#lo primero modifico los valores nulos
sss.age.fillna('Unknown', inplace=True)

In [210]:
#ahora voy a modificar todos los valores que no son los que yo quiero....esto seguro que se puede hacer de otra manera pero...
sss.age.replace(['40s','20s', 'Teen','60s', '18 months','30s', '50s','18 or 20',
       '12 or 13','Teens','8 or 10','� ', ' ', '30 or 36', '6�', '21 & ?','33 or 37', 'mid-30s', '7      &    31', ' 28',
       '20?', "60's", '16 to 18','Elderly','mid-20s', 'Ca. 33','21 or 26','>50','18 to 22', 'adult','? & 19', '9 months',
       '25 to 35','(adult)','25 or 28','? & 27','X', '"middle-age"', '13 or 18', '2 to 3 months', 'MAKE LINE GREEN', ' 43',
       '"young"','7 or 8','F', 'Both 11', '9 or 10', 'young','  ', 'A.M.', '?    &   14', '10 or 12', '31 or 33',
       '2�', '13 or 14'],
        ['40','20', 'aprox.15','60', '1.5','30', '50','19','12.5','aprox.15','9','Unknown', 'Unknown', '33', '6', 
         '21 & Unknown','35', 'aprox.30', '7&31', '28','20', "60", '17','aprox 75','aprox.20', '33','23',
         '50','20', 'aprox 40','Unknown & 19', '0.75','30','aprox.40','27','Unknown & 27','Unknown',
         'aprox 40', '16', '0.25', 'Unknown', '43','aprox25','7.5','Unknown', '11', '9.5', 'aprox.25','Unknown',
         'Unknown', 'Unknown & 14', '11', '32','2', '13.5'],
                        inplace=True)

In [216]:
sss.age.head()

0         57
1         11
2         48
3    Unknown
4    Unknown
Name: age, dtype: object

# 13. Proceso de limpieza de columna href:

-La columna herf solo tiene un valor nulo, pero luego revisandola me di cuanta que habia valores que no coincidian con enlaces a paginas web por lo que elaboré una función para sustituir esos valores también por unknown.

In [217]:
#nan_cols=sss.isna().sum()
#nan_cols[nan_cols>0]

In [218]:
mal=sss.href_formula[sss.href_formula.isna()].index
mal

Int64Index([3244], dtype='int64')

In [219]:
sss.href_formula.fillna('Unknown', inplace=True)
nan_cols=sss.isna().sum()
nan_cols[nan_cols>0]

injury                      28
fatal_(y/n)                539
time                      3354
species_                  2838
investigator_or_source      17
dtype: int64

In [220]:
def htpp(x):
    if re.search("^https?://",str(x)):
        return x
    else:
        return 'Unknown'
        

In [221]:
sss["href_formula"] = sss["href_formula"].apply(htpp)

In [222]:
sss.href_formula.head()

0    http://sharkattackfile.net/spreadsheets/pdf_di...
1    http://sharkattackfile.net/spreadsheets/pdf_di...
2    http://sharkattackfile.net/spreadsheets/pdf_di...
3    http://sharkattackfile.net/spreadsheets/pdf_di...
4    http://sharkattackfile.net/spreadsheets/pdf_di...
Name: href_formula, dtype: object

# 14.Proceso de limpieza columna fatal:

-Sustituyo los valores nulos por unknown.

-Unifico los valores únicos en tres caterorías, Y, N y UNKNOWN.

In [224]:
#nan_cols=sss.isna().sum()
#nan_cols[nan_cols>0]

In [225]:
sss.rename(columns = {'fatal_(y/n)':'fatality'}, inplace = True) #le cambio el nombre xq no se xq no me funcionaba

In [226]:
sss.columns

Index(['case_number', 'year', 'typess', 'country', 'area', 'location',
       'activity', 'name', 'sex_', 'age', 'injury', 'fatality', 'time',
       'species_', 'investigator_or_source', 'pdf', 'href_formula',
       'original_order'],
      dtype='object')

In [227]:
sss.fatality.fillna('UNKNOWN', inplace=True)#los modifico para que solo sea yes=Y no=N y Unknown, primero sustituyo los NAN

In [228]:
sss.fatality.replace(['N','Y','M','UNKNOWN','2017',' N','N ','y'],
                ['N','Y','UNKNOWN','UNKNOWN','UNKNOWN','N','N','Y'], inplace=True)

In [229]:
sss.fatality.unique()

array(['N', 'Y', 'UNKNOWN'], dtype=object)

# 15.Proceso limpieza columna investigator:
-En esta no me lio mucho, ni me interesa ni voy a modificar nada, los valores nulos por unknown y listo.

In [231]:
#nan_cols=sss.isna().sum()
#nan_cols[nan_cols>0]

In [232]:
sss.investigator_or_source.fillna('Unknown', inplace=True)

In [233]:
#INJURY
nan_cols=sss.isna().sum()
nan_cols[nan_cols>0]

injury        28
time        3354
species_    2838
dtype: int64

# 16.Proceso limpieza columna injury:

-Primero sustituyo los nulos por unknown.

-Esta columna tiene muchísimos valores únicos y mal definidos, por lo que creo una función que me permita agruparlos todos o al menos agruparlos en un rango menor de valores.

In [234]:
sss.injury.unique().tolist()[:10]


['No injury to occupant, outrigger canoe and paddle damaged',
 'Minor injury to left thigh',
 'Injury to left lower leg from surfboard skeg',
 'Minor injury to lower leg',
 'Lacerations to leg & hand shark PROVOKED INCIDENT',
 'No injury, board bitten',
 'FATAL',
 'Minor injury to foot. PROVOKED INCIDENT',
 'Lower left leg bitten',
 'Minor injury to foot']

In [235]:
sss.injury.fillna('Unknown', inplace=True)

In [236]:
nan_cols=sss.isna().sum()
nan_cols[nan_cols>0]

time        3354
species_    2838
dtype: int64

In [237]:

def limpiar_injury(x):
    dicc_injurys = {"No Injury":re.search(".*[Nn](o|O)\s*[Ii](njury|NJURY).*",str(x)),
                    "Minor Injury":re.search(".*[Mm](inor|INOR).*",str(x)),
                    "Fatal":re.search(".*[Ff](atal|ATAL).*",str(x)),
                    "Cuts":re.search(".*[Cc](uts|UTS).*",str(x)),
                    "Laceration":re.search(".*[Ll](aceration|ACERACION).*" or ".*[Aa](brasion|BRASION).*",str(x)),
                    "Bitten":re.search(".*[Bb](itten|ITTEN).*" or ".*[Bb](ite|ITE).*"or".*[Nn](ipped|IPPED).*",str(x)),
                    "Puncture wounds":re.search(".*[Pp](uncture|UNCTURE).*",str(x)),
                    "Injury in  leg":re.search(".*[Ii](njury|NJURY).*"and".*[Ll](eg|EG).*",str(x)),
                    "Injury in hand":re.search(".*[Ii](njury|NJURY).*"and".*[Hh](and|AND).*",str(x)),
                    "Injury in arm":re.search(".*[Ii](njury|NJURY).*" and ".*[Aa](rm|RM).*",str(x)),
                    "Injury in ankle":re.search(".*[Ii](njury|NJURY).*" and ".*[Aa](nkle|nkle).*",str(x)),
                    "Injury in foot":re.search(".*[Ii](njury|NJURY).*" and ".*[Ff](oot|OOT).*",str(x)),
                    "Injury in calf":re.search(".*[Ii](njury|NJURY).*" and ".*[Cc](alf|ALF).*",str(x)),
                    "Broken bones ":re.search(".*[Bb](roken|ROKEN).*" or ".*[Bb](ones|ONES).*",str(x)),
                    "Injury":re.search(".*[Ii](njury|NJURY).*",str(x)),
                    "Collision":re.search(".*[Cc](ollision|OLLISION).*" or ".*[Cc](ollided|OLLIDED).*",str(x)),
                    "Severe Injury":re.search(".*[Ss](evere|EVERE).*" or ".*[Ss](erius|ERIUS).*" and ".*[Ii](njury|NJURY).*",str(x))}

    
    for key,values in dicc_injurys.items():
        if values:
            return key
        
    return "Unknown"

In [238]:
sss["injury"] = sss["injury"].apply(limpiar_injury)

In [239]:
sss.injury.unique()

array(['No Injury', 'Minor Injury', 'Injury in  leg', 'Laceration',
       'Fatal', 'Bitten', 'Injury in arm', 'Unknown', 'Injury in hand',
       'Injury in ankle', 'Puncture wounds', 'Injury in foot',
       'Injury in calf', 'Cuts', 'Injury', 'Broken bones ',
       'Severe Injury', 'Collision'], dtype=object)

# 17.Proceso de limpieza columna time:

-Time es un dato con casi la mitad de los valores como nulos...si fuera por mi la borraba, uno porque no me sirve de nada para mi estudio e hipótsis y dos porque creo que una columna con la mitad de las valores nulos no puede ser utilizada para el estudio. No obstante no puedo borrarla por lo que simplemente sustituyo los nulos por Unknown  y creo una función para que me cambie todo lo que no sea estrictamente una hora(valor numérico) por unknown.

In [240]:
sss.time.fillna('Unknown', inplace=True)

In [241]:
#nan_cols=sss.isna().sum()
#nan_cols[nan_cols>0]

In [242]:
sss.time.head()

0            18h00
1    14h00  -15h00
2            07h45
3          Unknown
4          Unknown
Name: time, dtype: object

In [243]:
def times(x):
    if re.search("\d+h\d+",str(x)):
        return x
    else:
        return 'Unknown'
        
        

In [244]:
sss["time"] = sss["time"].apply(times)

In [245]:
sss.time.head()

0            18h00
1    14h00  -15h00
2            07h45
3          Unknown
4          Unknown
Name: time, dtype: object

# 18.Proceso de limpieza columna species:

-Sustituyo los valores nulos por Unknown.

-Reduzco el número de valores únicos a través de una función.

In [246]:
#species
sss.species_.fillna('Unknown', inplace=True)

In [247]:
def limpiar_species(x):
    dicc_especies = {
            "White shark":re.search(".*[Ww](hite|HITE).*",str(x)),
            "Tiger shark":re.search(".*[Tt](iger|IGER).*",str(x)),
            "Lemon shark":re.search(".*[Ll](emon|EMON).*",str(x)),
            "Hammerhead shark":re.search(".*[Hh](ammerhead|AMMERHEAD).*",str(x)),
            "Bull shark":re.search(".*[Bb](ull|ULL).*",str(x)),
            "Blue shark":re.search(".*[Bb](lue|LUE).*",str(x)),
            "Silvertip shark":re.search(".*[Ss](ilvertip|ILVERTIP).*",str(x)),
            "Nurse shark":re.search(".*[Nn](urse|URSE).*",str(x)),
            "Whaler shark":re.search(".*[Ww](haler|HALER).*",str(x)),
            "Blacktip shark":re.search(".*[Bb](lacktip|LACKTIP).*",str(x)),
            "Mako shark":re.search(".*[MM](ako|AKO).*",str(x)),
            "Sand shark":re.search(".*[Ss](and|AMD).*",str(x)),
            "Wobbegong shark":re.search(".*[Ww](obbegong|OBBEGONG).*",str(x)),
            "Galapagos shark":re.search(".*[Gg](alapagos|ALAPAGOS).*",str(x)),
            "Grey shark":re.search(".*[Gg](rey|REY).*",str(x)),
            "Leopard shark":re.search(".*[Ll](eopard|EOPARD).*",str(x)),
            "Zambesi shark":re.search(".*[Zz](ambesi|AMBESI).*",str(x)),
            "Blacktail shark":re.search(".*[Bb](lacktail|LACKTAIL).*",str(x)),
            "Red shark":re.search(".*[Rr](ed|ED).*",str(x)),
            "Dusky shark":re.search(".*[Dd](usky|USKY).*",str(x)),
            "Raggedtooth shark":re.search(".*[Rr](aggedtooth|AGGEDTOOTH).*",str(x)),
            "Spinner shark":re.search(".*[Ss](pinner|PINNER).*",str(x)),
            "Cow shark":re.search(".*[Cc](ow|OW).*",str(x)),
            "Porbeagle shark":re.search(".*[Pp](orbeagle|ORBEAGLE).*",str(x)),
            "Caribbean reef shark":re.search(".*[Cc](aribbean|ARIBBEAN).*",str(x)),
            "Sandbar shark":re.search(".*[Ss](and|AND).*",str(x)),
            "Silky shark":re.search(".*[Ss](ilky|ILKY).*",str(x)),
            "Zambezi shark":re.search(".*[Zz](ambezi|AMBEZI).*",str(x)),
            "Sevengill shark":re.search(".*[Ss](evengill|EVENGILL).*",str(x)),
            "Copper shark":re.search(".*[Cc](opper|OPPER).*",str(x)),
            "Angel shark":re.search(".*[Aa](ngel|NGEL)\s",str(x)),
            "Salmon shark":re.search(".*[Ss](almon|ALMON).*",str(x)),
            "Goblin shark":re.search(".*[Gg](oblin|OBLIN).*",str(x)),
            "Thresher shark":re.search(".*[Tt](hresher|HRESHER).*",str(x)),
            "Dogfish shark":re.search(".*[Dd](ogfish|OGFISH).*",str(x)),
            "Involvement not confirmed":re.search("[^.?!]*involvement[^.?!]*",str(x))}
    for key,values in dicc_especies.items():
        if values:
            return key

    return "other_specie"


In [248]:
sss["species_"] = sss["species_"].apply(limpiar_species)

In [249]:
sss.species_.unique()

array(['White shark', 'other_specie', 'Tiger shark', 'Lemon shark',
       'Bull shark', 'Grey shark', 'Nurse shark',
       'Involvement not confirmed', 'Wobbegong shark', 'Blacktip shark',
       'Galapagos shark', 'Blue shark', 'Spinner shark',
       'Porbeagle shark', 'Caribbean reef shark', 'Sevengill shark',
       'Angel shark', 'Dogfish shark', 'Mako shark', 'Whaler shark',
       'Silky shark', 'Hammerhead shark', 'Raggedtooth shark',
       'Goblin shark', 'Sand shark', 'Cow shark', 'Salmon shark',
       'Zambesi shark', 'Thresher shark', 'Dusky shark', 'Red shark',
       'Copper shark', 'Leopard shark', 'Silvertip shark',
       'Zambezi shark', 'Blacktail shark'], dtype=object)

In [250]:
nan_cols=sss.isna().sum()
nan_cols[nan_cols>0]

Series([], dtype: int64)

In [251]:
sss.to_csv(r'C:\Users\Ari\desktop\ironhack\Proyec-1-Pandas-Shark-attacks\cleanattacks.csv', index=False)